In [ ]:
# When running for the first time, run in terminal to set up the environment:
    # python -m venv env
    # Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope Process
    # .\env\Scripts\activate
    # pip install -r requirements.txt

In [2]:
import os
import glob
import soundfile as sf
import torch
import pandas as pd
from audiobox_aesthetics.infer import initialize_predictor

c:\GitHub\audio-aesthetics\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def main():
    audio_dir = r"C:\GitHub\audio-aesthetics\assets"
    audio_files = glob.glob(os.path.join(audio_dir, "*.mp3"))
    
    predictor = initialize_predictor()
    
    batch_data = []
    for fpath in audio_files:
        data, sr = sf.read(fpath)
        audio_tensor = torch.tensor(data, dtype=torch.float32).T
        batch_data.append({"path": audio_tensor, "sample_rate": sr, "filename": fpath})
    
    # Forward pass over all files
    predictions = predictor.forward(batch_data)
    
    results = []
    for pred, info in zip(predictions, batch_data):
        ce_val = pred.get("CE", None)
        cu_val = pred.get("CU", None)
        pc_val = pred.get("PC", None)
        pq_val = pred.get("PQ", None)
        
        file_name = os.path.basename(info["filename"])
        results.append([
            file_name,
            ce_val,
            cu_val,
            pc_val,
            pq_val
        ])
    
    # Rename columns in the final DataFrame
    df = pd.DataFrame(results, columns=[
        "File",
        "Content Enjoyment (CE)",
        "Content Usefulness (CU)",
        "Production Complexity (PC)",
        "Production Quality (PQ)"
    ])
    print(df)
    df.to_csv("aesthetics_results.csv", index=False)

if __name__ == "__main__":
    main()


c:\GitHub\audio-aesthetics\env\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
c:\GitHub\audio-aesthetics\env\lib\site-packages\torch\nn\functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


                           File  Content Enjoyment (CE)  \
0          playlist_bernina.mp3                6.759501   
1          playlist_citroen.mp3                5.785450   
2          playlist_melinda.mp3                6.228623   
3  playlist_natural-trainer.mp3                6.344067   
4      playlist_parmareggio.mp3                6.677727   
5          playlist_repower.mp3                6.305100   
6         playlist_tenderly.mp3                5.872531   
7        playlist_valfrutta.mp3                6.015845   
8           playlist_yakult.mp3                6.354249   

   Content Usefulness (CU)  Production Complexity (PC)  \
0                 5.221872                    6.496495   
1                 6.332826                    5.257134   
2                 6.047773                    6.500406   
3                 5.609169                    6.538504   
4                 5.640616                    6.699841   
5                 5.676331                    6.908421   
6  

In [7]:
import importlib.util
print(importlib.util.find_spec("safetensors") is not None)

True
